<a href="https://colab.research.google.com/github/kazuki-de/git_reserch_code/blob/master/IMSAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-beta1
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D, Activation , Softmax, MaxPool2D,UpSampling2D
from tensorflow.keras import Model
import datetime
#import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image

In [2]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd  /content/drive/My Drive/logs

/content/drive/My Drive/logs


In [0]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="/content/drive/My Drive/logs/gradient_tape", histogram_freq=1)

In [0]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu',)
        
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
        

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model1 = MyModel()

In [0]:
def kl_divergence(q_logit, p_logit):
    qlogq = tf.reduce_sum(q_logit * tf.math.log(q_logit), axis=1)
    qlogp = tf.reduce_sum(q_logit * tf.math.log(p_logit), axis=1)
    return qlogq - qlogp
def get_unit_vector(v):
    return  v / (tf.sqrt(tf.reduce_sum(v ** 2, axis=None,keepdims=True) + 1e-16))

In [0]:
def VAT_KL(input_tensor, network, xi=10, epsilon=1.0, weight=1.0, num_approximation=1, clip_value_min=1e-16, dtype=tf.float64):
    
    #配列内の数値の最大値が第二変数になるように置き換える
    #clipped = lambda x: tf.maximum(x, clip_value_min)
    #axis_without_batch_size は (1,2,3)となる
    #axis_without_batch_size = tuple(range(1,len(input_tensor.get_shape())))
    
    #if len(axis_without_batch_size) == 1:
    #    axis_without_batch_size = axis_without_batch_size[0]
        
    #normalized = lambda x: x / clipped(tf.norm(x, axis=None))
    #normalized = lambda x: x / clipped(tf.norm(x, axis=None,keepdims=True))
    #normalized = lambda x: x / tf.norm(x, axis=None,keepdims=True)
    plain_softmax = network(input_tensor)
    
    noplain_softmax = Activation("softmax")(plain_softmax)
    
    #適当な単位ベクトルにxi=10を掛けた数:pertubationの作成
    perturbation = xi * get_unit_vector(tf.random.normal(shape=tf.shape(input_tensor), dtype=dtype))
    #print(perturbation.numpy())
    
    for i in range(num_approximation):
        
        softmax_accommodating_perturbation = network(input_tensor+perturbation)
        
        softmax_accommodating_perturbation_af = Activation("softmax")(softmax_accommodating_perturbation)
        # ノイズを足した配列とノーマルな配列がCNNで出力された結果のKL距離を求める
        dist = kl_divergence(noplain_softmax,softmax_accommodating_perturbation_af)
        #cross_entropy_accommodating_perturbation = -tf.reduce_sum(plain_softmax * tf.math.log(clipped(softmax_accommodating_perturbation))) * weight
        adversarial_direction = tf.gradients(dist, [perturbation])[0]
        pertubation = tf.stop_gradient(adversarial_direction)
    #print("ok")    
    
    pertubation = epsilon * get_unit_vector(pertubation)
    corrent_softmax = Activation("softmax")(network(input_tensor)) 
    vat_softmax = Activation("softmax")(network(input_tensor + perturbation))
    loss =  -kl_divergence(corrent_softmax,vat_softmax)
    return  loss

In [0]:
def compute_entropy(p):
    return - tf.reduce_sum(p*tf.math.log(p+1e-16),axis = 1)

def compute_marginal_entropy(p_batch):
    return  compute_entropy(tf.reduce_mean(p_batch))

In [0]:
#損失関数
loss_object = tf.keras.losses.CategoricalCrossentropy()
#最適化手法
optimizer = tf.keras.optimizers.Adam()

#評価関数
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_loss1 = tf.keras.metrics.Mean(name='train_loss')
train_loss2 = tf.keras.metrics.Mean(name='train_loss')


train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')

test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [0]:
#ミニバッチ学習==1epocでn枚の画像からx枚ずつ取り出しn/x回学習する(重みを更新していく)手法
#バッチ学習 == 1epocでn枚の画像から1枚ずつ取り出しn回学習する（重みを更新していく）手法
#オンライン学習 ==1epocでn枚の画像からn枚全てを用い学習する(重みを更新していく)手法　#メモリ消費大
@tf.function
def train_step(image,label,batch_size):
    with tf.GradientTape() as tape:
        p = Activation('softmax')(model1(image))
        hy_ = compute_entropy(p)
        hy = tf.reduce_mean(hy_)
        p_ave = tf.reduce_mean(p,axis=0)
        hy_x = -tf.reduce_sum(p_ave*tf.math.log(p_ave+1e-16))
        #print(hy)
        #hy_x = tf.reduce_sum(compute_entropy(p))/batch_size
        #print(hy_x)
        Rsut = -tf.reduce_sum(VAT_KL(image,model1))/batch_size
        #print(Rsut)
        loss = Rsut - 0.2*((4.0)*hy-hy_x)
    gradients = tape.gradient(loss, model1.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model1.trainable_variables))
    
    train_loss(loss)
    #train_loss1(Rsut)
    #train_loss2(0.2*((4.0)*hy-hy_x))
    #train_loss(loss)
    #train_loss
    
    
        

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(100)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(100)

In [12]:
%pip install munkres
from munkres import Munkres, print_matrix

In [0]:
def compute_accuracy(y_pred, y_t,tot_cl=10):
    # compute the accuracy using Hungarian algorithm
    m = Munkres()
    mat = np.zeros((tot_cl, tot_cl))
    for i in range(tot_cl):
        for j in range(tot_cl):
            mat[i][j] = np.sum(np.logical_and(y_pred == i, y_t == j))
    indexes = m.compute(-mat)

    corresp = []
    for i in range(tot_cl):
        corresp.append(indexes[i][1])

    pred_corresp = [corresp[predicte] for predicted in y_pred]
    acc = np.sum(pred_corresp == y_t) / float(len(y_t))
    return acc

In [14]:
epoc =20
for i in range(epoc):
    for image,label in train_ds:
        train_step(image,label,100)
    #print( train_loss1.result())  
    #print( train_loss2.result())
    print( train_loss.result())
    #print(compute_accuracy(np.argmax(Activation('softmax')(model1(x_train)).astype(np.float32),y_train.astype(np.float32).,10))
    

tf.Tensor(-1.3813661, shape=(), dtype=float32)
tf.Tensor(-1.3814718, shape=(), dtype=float32)
tf.Tensor(-1.3815118, shape=(), dtype=float32)
tf.Tensor(-1.3815318, shape=(), dtype=float32)
tf.Tensor(-1.3815409, shape=(), dtype=float32)
tf.Tensor(-1.3815087, shape=(), dtype=float32)
tf.Tensor(-1.3814857, shape=(), dtype=float32)
tf.Tensor(-1.3814684, shape=(), dtype=float32)
tf.Tensor(-1.381455, shape=(), dtype=float32)
tf.Tensor(-1.3814498, shape=(), dtype=float32)
tf.Tensor(-1.381485, shape=(), dtype=float32)
tf.Tensor(-1.3815142, shape=(), dtype=float32)
tf.Tensor(-1.381539, shape=(), dtype=float32)
tf.Tensor(-1.3815602, shape=(), dtype=float32)
tf.Tensor(-1.3815786, shape=(), dtype=float32)
tf.Tensor(-1.3815947, shape=(), dtype=float32)
tf.Tensor(-1.3816088, shape=(), dtype=float32)
tf.Tensor(-1.3816215, shape=(), dtype=float32)
tf.Tensor(-1.3816327, shape=(), dtype=float32)
tf.Tensor(-1.3816314, shape=(), dtype=float32)


In [17]:
Activation('softmax')(model1(x_train[1:10]))

<tf.Tensor: id=25805, shape=(9, 10), dtype=float64, numpy=
array([[0.1000011 , 0.09999994, 0.0999999 , 0.09999994, 0.10000285,
        0.09999996, 0.09999986, 0.09999988, 0.09999796, 0.0999986 ],
       [0.1000011 , 0.09999994, 0.0999999 , 0.09999994, 0.10000285,
        0.09999996, 0.09999986, 0.09999988, 0.09999796, 0.0999986 ],
       [0.1000011 , 0.09999994, 0.0999999 , 0.09999994, 0.10000285,
        0.09999996, 0.09999986, 0.09999988, 0.09999796, 0.0999986 ],
       [0.1000011 , 0.09999994, 0.0999999 , 0.09999994, 0.10000285,
        0.09999996, 0.09999986, 0.09999988, 0.09999796, 0.0999986 ],
       [0.1000011 , 0.09999994, 0.0999999 , 0.09999994, 0.10000285,
        0.09999996, 0.09999986, 0.09999988, 0.09999796, 0.0999986 ],
       [0.1000011 , 0.09999994, 0.0999999 , 0.09999994, 0.10000285,
        0.09999996, 0.09999986, 0.09999988, 0.09999796, 0.0999986 ],
       [0.1000011 , 0.09999994, 0.0999999 , 0.09999994, 0.10000285,
        0.09999996, 0.09999986, 0.09999988, 0.09999